Problem Set 1

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

We delete clients who leave the company (Churn=No)

In [5]:
df = pd.read_csv (r'customers.csv') #import data
negative_churn = df.Churn=='No' #find who leave the company (churn) 
df = df.drop(df[negative_churn].index) #delete them from the DataFrame
df.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
13,0280-XJGEX,Male,0,No,No,49,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes
18,4190-MFLUW,Female,0,Yes,Yes,10,Yes,No,DSL,No,...,Yes,Yes,No,No,Month-to-month,No,Credit card (automatic),55.20,528.35,Yes
20,8779-QRDMV,Male,1,No,No,1,No,No phone service,DSL,No,...,Yes,No,No,Yes,Month-to-month,Yes,Electronic check,39.65,39.65,Yes
22,1066-JKSGK,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.15,20.15,Yes
26,6467-CHFZW,Male,0,Yes,Yes,47,Yes,Yes,Fiber optic,No,...,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.35,4749.15,Yes
27,8665-UTDHZ,Male,0,Yes,Yes,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,30.20,30.2,Yes


In [6]:
df["TotalCharges"] = pd.to_numeric(df.TotalCharges) # in order to convert total charges in floting data
print(df.TotalCharges)

2        108.15
4        151.65
5        820.50
8       3046.05
13      5036.30
         ...   
7021     727.80
7026     403.35
7032      75.75
7034    6886.25
7041     306.60
Name: TotalCharges, Length: 1869, dtype: float64


In [7]:
number_tot = len(negative_churn) #total number of clients
number_positive_churn = len(df) #number of clients who do NOT leave the company 
perc_positive_tot=number_positive_churn/number_tot
print('Percentage of clients who do NOT leave the company (on the total) = %.3f' % perc_positive_tot)
print('Percentage of clients who leave the company (on the total) = %.3f' % (1-perc_positive_tot))

Percentage of clients who do NOT leave the company (on the total) = 0.265
Percentage of clients who leave the company (on the total) = 0.735


In [8]:
female = df.gender=='Female' #find woman
n_female = np.sum(female) #count the True values
n_male = len(female)-n_female
perc_female = n_female/number_tot
perc_male = n_male/number_tot
data = {'female':perc_female, 'male':perc_male}
print(type(data))
#sns.barplot(x='gender', y='percentage', data=data)


<class 'dict'>
